# Graph Neural Networks
## What are Graph Neural Networks (GNNs)?

In [24]:
#import the basics
import os
import torch
import torch_geometric
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from networkx.classes import number_of_nodes
%matplotlib inline

In [25]:
torch_geometric.__version__

'2.6.1'

In [26]:
# Let's verify what device we are working with
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("You are using device: %s" % device)

You are using device: cuda


Graph Neural Networks are a type of "geometric deep learning" models that use pairwise message passing. They typically have an architecture consisting of 3 types of layers. From [wikipedia](https://en.wikipedia.org/wiki/Graph_neural_network):
1. Permutation equivariant: a permutation equivariant layer maps a representation of a graph into an updated representation of the same graph. In the literature, permutation equivariant layers are implemented via **pairwise message passing between graph nodes**. Intuitively, in a message passing layer, nodes update their representations by aggregating the messages received from their immediate neighbours. As such, each message passing layer increases the receptive field of the GNN by one hop.
2. Local pooling: a local pooling layer coarsens the graph via downsampling. Local pooling is used to increase the receptive field of a GNN, in a similar fashion to pooling layers in convolutional neural networks. Examples include k-nearest neighbours pooling, top-k pooling, and self-attention pooling.
3. Global pooling: a global pooling layer, also known as readout layer, provides fixed-size representation of the whole graph. The global pooling layer must be permutation invariant, such that permutations in the ordering of graph nodes and edges do not alter the final output. Examples include element-wise sum, mean or maximum.

## Attributes
- [T]he preprocessing step first
“squashes” the graph structured data into a vector of reals and
then deals with the preprocessed data using a list-based data
processing technique. However, important information, e.g., the
topological dependency of information on each node may be
lost during the preprocessing stage and the final result may depend, in an unpredictable manner, on the details of the preprocessing algorith [1] **GNNS preserve the structure of the graph it is based on.**
- It will be shown that the GNN
is an extension of both recursive neural networks and random
walk models and that it retains their characteristics. The model
extends recursive neural networks since it can process a more
general class of graphs including cyclic, directed, and undirected graphs, and it can deal with node-focused applications
without any preprocessing steps. The approach extends random
walk theory by the introduction of a learning algorithm and by
enlarging the class of processes that can be modeled. [1]
- Weights are shared across layer structures

### What is message passing?
From [wikipedia](https://en.wikipedia.org/wiki/Graph_neural_network#Message_passing_layers):
<br>
![img](./img/notebook/messagePassing.png)

## Computation Graph
"The neighbour of a node defines its computation graph" - @12:34 https://www.youtube.com/watch?v=JtDgmmQ60x8&ab_channel=AntonioLonga



# Data
Heterogeneous graphs are perfect for recommendation systems. Let's examine a data set from pytorch geometric to understand some basics about the data.

### Datasets:
"AmazonBook" - A subset of the AmazonBook rating dataset from the "LightGCN: Simplifying and Powering Graph Convolution Network for Recommendation" paper.

"MovieLens" - A heterogeneous rating dataset, assembled by GroupLens Research from the MovieLens website, consisting of nodes of type "movie" and "user". User ratings for movies are available as ground truth labels for the edges between the users and the movies ("user", "rates", "movie").

In [59]:
from torch_geometric.datasets import AmazonBook, MovieLens
from torch_geometric.transforms import Compose, ToDevice, ToUndirected

transform = Compose([ToDevice(device), ToUndirected()])

# amazon_dataset = AmazonBook(root="./data/AmazonBook")
movielens_dataset = MovieLens(root="./data/MovieLens", transform=transform, model_name='all-MiniLM-L6-v2')

print(f"Dataset: {movielens_dataset}")
print(f"Number of graphs in dataset: {len(movielens_dataset)}")
print(f"Number of features of dataset: {movielens_dataset.num_features}")

Dataset: MovieLens()
Number of graphs in dataset: 1
Number of features of dataset: {'movie': 404, 'user': 0}


In [60]:
data = movielens_dataset[0]
print(f"data = Dataset[0]: {data}")

print(f"Number of node features of data: {data.num_features}")
print(f"Number of edge features of data: {data.num_edge_features}")
print(f"Number of nodes of data: {data.num_nodes}")
print(f"Number of edges of data: {data.num_edges}")
print(f"data is directed?: {data.is_directed()}")
print(f"data has isolated nodes?: {data.has_isolated_nodes()}")
print(f"data contains self loops?: {data.has_self_loops()}")
print(f"data node types: {data.node_types}")
print(f"data edge types: {data.edge_types}")
print(f"data is on device: {'CUDA' if data.is_cuda else 'CPU'}")

data = Dataset[0]: HeteroData(
  movie={ x=[9742, 404] },
  user={ num_nodes=610 },
  (user, rates, movie)={
    edge_index=[2, 100836],
    edge_label=[100836],
    time=[100836],
  },
  (movie, rev_rates, user)={
    edge_index=[2, 100836],
    edge_label=[100836],
    time=[100836],
  }
)
Number of node features of data: {'movie': 404, 'user': 0}
Number of edge features of data: {('user', 'rates', 'movie'): 0, ('movie', 'rev_rates', 'user'): 0}
Number of nodes of data: 10352
Number of edges of data: 201672
data is directed?: False
data has isolated nodes?: True
data contains self loops?: False
data node types: ['movie', 'user']
data edge types: [('user', 'rates', 'movie'), ('movie', 'rev_rates', 'user')]
data is on device: CUDA


In [61]:
data.validate(raise_on_error=True)

True

In [30]:
# print(f"Number of training edges: {data['user','rates','book'].edge_index.shape[1]}")
# print(f"Number of testing edges: {data['user','rates','book'].edge_label_index.shape[1]}")

In [31]:
# print(f"{torch.min(data['user', 'rates', 'book'].edge_index[0])}")
# print(f"{torch.max(data['user', 'rates', 'book'].edge_index[0])}")

Because there are no features on the edges or the nodes, the connections (edges) themselves are what we are training on.

- Link Prediction on MovieLens.ipynb - https://colab.research.google.com/drive/1xpzn1Nvai1ygd_P5Yambc_oe4VBPK_ZT?usp=sharing#scrollTo=JMGYv83WzSRr
- [1] Neural Graph Collaborative Filtering - https://dl.acm.org/doi/pdf/10.1145/3331184.3331267

In [62]:
torch_geometric.seed_everything(1234)

## Naive Graph Neural Network



In [279]:
from torch_geometric.nn import MLP, MessagePassing, Linear
from torch.nn import Parameter
from torch.nn import Embedding
from torch.nn import functional as F

"""
    GNN layer consists of:
        1.
        2.
        3.
"""

class GNNLayer(MessagePassing):
    def __init__(self, num_of_nodes_sampled=30, hidden_channels=128, aggr='mean'):
        super(GNNLayer, self).__init__(aggr=aggr)
        self.W1 = Parameter(torch.Tensor(hidden_channels))
        self.W2 = Parameter(torch.Tensor(hidden_channels))

        self.p_ui = 1/torch.sqrt(num_of_nodes_sampled*num_of_nodes_sampled)

        self.reset_parameters()

    def reset_parameters(self):
        torch.nn.init.xavier_uniform_(self.W1)
        torch.nn.init.xavier_uniform_(self.W2)

    def forward(self, edge_index, x):

        return self.propagate(edge_index=edge_index, x=x)

    def message(self,x_j, x_i):
        """
            For a connected user-movie pair (i, j), we define the message from j to i
                m[i<-j] = 1/sqrt(num_of_users*num_of_items) * (W1*ei + W2(ei element-wise-product eu))

        :param x_j:
        :param x_i:
        :return:
        """
        return self.message_function(torch.concat([x_i, x_j], dim=1))

    def update(self, aggr_out):
        return self.update_function(aggr_out)

class GNN(torch.nn.Module):
    def __init__(self, hidden_channels=1280, aggr='mean'):
        super(GNN, self).__init__()
        self.gnn_layer_1 = GNNLayer(hidden_channels=hidden_channels, aggr=aggr)

    def forward(self, edge_index, x):
        out = self.gnn_layer_1(edge_index, x)

        edge_feat_user = out[edge_index[0]]
        edge_feat_movie = out[edge_index[1]]

        return (edge_feat_user * edge_feat_movie).sum(dim=1)


In [268]:
from torch_geometric.transforms import RandomLinkSplit

# lets split the data into training, validation, and test sets
transform = RandomLinkSplit(
    edge_types=('user', 'rates', 'movie'),
    rev_edge_types=('movie', 'rev_rates', 'user')
)

train_data, val_data, test_data = transform(data)

In [269]:
train_data

HeteroData(
  movie={ x=[9742, 404] },
  user={ num_nodes=610 },
  (user, rates, movie)={
    edge_index=[2, 70586],
    time=[70586],
    edge_label=[141172],
    edge_label_index=[2, 141172],
  },
  (movie, rev_rates, user)={
    edge_index=[2, 70586],
    edge_label=[70586],
    time=[70586],
  }
)

In [270]:
val_data

HeteroData(
  movie={ x=[9742, 404] },
  user={ num_nodes=610 },
  (user, rates, movie)={
    edge_index=[2, 70586],
    time=[70586],
    edge_label=[20166],
    edge_label_index=[2, 20166],
  },
  (movie, rev_rates, user)={
    edge_index=[2, 70586],
    edge_label=[70586],
    time=[70586],
  }
)

In [271]:
test_data

HeteroData(
  movie={ x=[9742, 404] },
  user={ num_nodes=610 },
  (user, rates, movie)={
    edge_index=[2, 80669],
    time=[80669],
    edge_label=[40334],
    edge_label_index=[2, 40334],
  },
  (movie, rev_rates, user)={
    edge_index=[2, 80669],
    edge_label=[80669],
    time=[80669],
  }
)

In [272]:
print(f"train data # of nodes: {train_data.num_nodes}")
print(f"train data # of edges: {train_data.num_edges}")
print(f"train data # of user nodes: {train_data['user'].num_nodes}")
print(f"train data # of movie nodes: {train_data['movie'].num_nodes}")
print(f"train data # of 'user', 'rates', 'movies' edges: {train_data['user','rates','movie'].num_edges}")
print(f"train data # of 'movie', 'rev_rates', 'user' edges: {train_data['movie','rev_rates','user'].num_edges}")
print(f"train data # of 'user', 'rates', 'movies' edge_label size: {train_data['user','rates','movie'].edge_label.size()}")
print(f"train data # of 'user', 'rates', 'movies' edge_label_index size: {train_data['user','rates','movie'].edge_label_index.size()}")
print(f"train data # of 'user', 'rates', 'movies' edge_label first 5: {train_data['user','rates','movie'].edge_label[0:5]}")
print(f"train data # of 'user', 'rates', 'movies' edge_label_index first 5: {train_data['user','rates','movie'].edge_label_index[0,0:5]}")
print(f"train data # of 'user', 'rates', 'movies' edge_label_index first 5: {train_data['user','rates','movie'].edge_label_index[1,0:5]}")
print(f"train data # of 'user', 'rates', 'movies' edge_label min: {train_data['user','rates','movie'].edge_label.min()}")
print(f"train data # of 'user', 'rates', 'movies' edge_label max: {train_data['user','rates','movie'].edge_label.max()}")
print(f"train data size of edge_index first 5: {train_data['user','rates','movie'].edge_index[0,0:5]}")
print(f"train data size of edge_index first 5: {train_data['user','rates','movie'].edge_index[1,0:5]}")
print(f"train data size of edge_index: {train_data['user','rates','movie'].edge_index.size()}")
print(f"train data size of edge_label_index: {train_data['user','rates','movie'].edge_label_index.size()}")
70586*2

train data # of nodes: 10352
train data # of edges: 141172
train data # of user nodes: 610
train data # of movie nodes: 9742
train data # of 'user', 'rates', 'movies' edges: 70586
train data # of 'movie', 'rev_rates', 'user' edges: 70586
train data # of 'user', 'rates', 'movies' edge_label size: torch.Size([141172])
train data # of 'user', 'rates', 'movies' edge_label_index size: torch.Size([2, 141172])
train data # of 'user', 'rates', 'movies' edge_label first 5: tensor([4, 5, 4, 4, 5], device='cuda:0')
train data # of 'user', 'rates', 'movies' edge_label_index first 5: tensor([297, 198, 231,  70, 407], device='cuda:0')
train data # of 'user', 'rates', 'movies' edge_label_index first 5: tensor([3568, 3565, 3505,  133, 8541], device='cuda:0')
train data # of 'user', 'rates', 'movies' edge_label min: 0
train data # of 'user', 'rates', 'movies' edge_label max: 6
train data size of edge_index first 5: tensor([297, 198, 231,  70, 407], device='cuda:0')
train data size of edge_index first 5

141172

In [283]:
from torch_geometric.loader import LinkNeighborLoader, NeighborLoader

# Define a mini-batch loader
num_samples = [30]
num_hops = 2
batch_size = 500

edge_type = ('user', 'rates', 'movie')
edge_label_index = train_data['user', 'rates', 'movie'].edge_label_index # edge indices for which neighbors are sampled to create mini-batch
edge_label= train_data['user', 'rates', 'movie'].edge_label

loader = LinkNeighborLoader(
    train_data,
    num_neighbors= num_samples,
    batch_size=batch_size,
    edge_label_index=(edge_type, edge_label_index),
    edge_label=edge_label,
    neg_sampling_ratio=2,
    shuffle=True
)

# loader = NeighborLoader(
#     train_data,
#     num_neighbors=num_samples * num_hops,
#     batch_size=batch_size
# )

In [284]:
sampled_data = next(iter(loader))
print(sampled_data)

HeteroData(
  movie={
    x=[4301, 404],
    n_id=[4301],
    num_sampled_nodes=[2],
  },
  user={
    num_nodes=608,
    n_id=[608],
    num_sampled_nodes=[2],
  },
  (user, rates, movie)={
    edge_index=[2, 10643],
    time=[10643],
    edge_label=[1500],
    edge_label_index=[2, 1500],
    e_id=[10643],
    num_sampled_edges=[1],
    input_id=[500],
  },
  (movie, rev_rates, user)={
    edge_index=[2, 14838],
    edge_label=[14838],
    time=[14838],
    e_id=[14838],
    num_sampled_edges=[1],
  }
)


In [285]:
len(loader)

283

In [293]:
print(sampled_data['movie']['num_sampled_nodes'])
print(sampled_data['user']['num_sampled_nodes'])

[1357, 2944]
[551, 57]


In [286]:
print(len(sampled_data["user", "rates", "movie"]['edge_label']))
print(sampled_data["user", "rates", "movie"].edge_label_index[:,0:5])
print(sampled_data["user", "rates", "movie"].edge_label[0:5])

1500
tensor([[398, 196, 297, 272, 105],
        [305, 228, 952, 205, 224]])
tensor([4, 5, 1, 4, 1])


In [241]:
torch.concat([torch.tensor([[1,2],[3,4]]),torch.tensor([[5,6],[7,8]])], dim=1)

tensor([[1, 2, 5, 6],
        [3, 4, 7, 8]])

In [287]:
num_of_epochs = 10
model = GNN()
model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

print(model)

GNN(
  (gnn_layer_1): GNNLayer()
)


In [288]:
def train_model(model, batch):
    optimizer.zero_grad()

    batch.to(device)

    target = batch['user', 'rates', 'movie'].edge_label.to(device).to(torch.float64)

    pred = model(batch['user','rates', 'movie'].edge_label_index, batch["movie"].x)

    loss = F.binary_cross_entropy_with_logits(pred, target)
    loss.backward()
    optimizer.step()

    return pred, loss

In [277]:
pred, loss = train_model(model, sampled_data)

x_i size: torch.Size([1500, 404])


In [266]:
torch.nn.functional.softmax(pred, dim=0).max()

tensor(1., device='cuda:0', grad_fn=<MaxBackward1>)

In [282]:
import tqdm
from torch.nn import functional as F

model.train()
for epoch in range(num_of_epochs):
    total_loss = total_examples = 0.0
    for batch in tqdm.tqdm(loader):
        pred, loss = train_model(model, batch)
        total_loss += float(loss) * pred.numel()
        total_examples += pred.numel()
    print(f"Epoch: {epoch:03d}, loss: {total_loss/total_examples:.4f}")


100%|██████████| 283/283 [00:04<00:00, 59.10it/s]


Epoch: 000, loss: -276054893.4609


100%|██████████| 283/283 [00:04<00:00, 64.23it/s]


Epoch: 001, loss: -6965468880.1423


100%|██████████| 283/283 [00:04<00:00, 64.58it/s]


...

Epoch: 007, loss: -2346320333454.2905


100%|██████████| 283/283 [00:04<00:00, 64.39it/s]


Epoch: 008, loss: -3637393403807.4053


100%|██████████| 283/283 [00:04<00:00, 62.57it/s]

Epoch: 009, loss: -5352262381109.0234
